<a href="https://colab.research.google.com/github/EXCALOFRIO/FENCINGLY.AI/blob/main/EntrenarTensorflowEsgrimav3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## IMPORTS

In [1]:
import os
import json
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from datetime import datetime
import logging
import shutil
!apt install unrar
!pip install rarfile

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unrar is already the newest version (1:6.1.5-1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [2]:
!nvidia-smi

Thu Feb 22 13:16:46 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
from rarfile import RarFile

# Ruta del archivo RAR
ruta_rar = "/content/drive/MyDrive/TFG/JSON_FORMADOS.rar"

# Ruta donde se desea descomprimir el archivo
ruta_destino = "/content/DATA_JSON/"

# Extraer el archivo RAR en la ruta especificada
with RarFile(ruta_rar) as rar:
  rar.extractall(ruta_destino)


## PREPARAR DATOS

In [5]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.get_logger().setLevel(logging.ERROR)


In [ ]:
def cargar_datos(directorio, num_frames=100):
    datos = []
    for archivo_json in os.listdir(directorio):
        ruta_json = os.path.join(directorio, archivo_json)
        with open(ruta_json) as f:
            datos_json = json.load(f)
            secuencia = np.zeros((num_frames, 2, 75))  # Inicializar matriz para la secuencia
            for i, frame in enumerate(datos_json['frames'][:num_frames]):
                poses = frame['poses'][:2]  # Tomar solo las primeras 2 poses
                for j, pose in enumerate(poses):
                    keypoints = pose['pose_keypoints_2d'][:75]  # Tomar solo los primeros 75 keypoints
                    secuencia[i, j, :] = keypoints
            datos.append(secuencia)
    return datos


def dividir_datos(datos, porcentaje_entrenamiento):
    cantidad_entrenamiento = int(porcentaje_entrenamiento * len(datos))
    indices = np.random.permutation(len(datos))
    datos_entrenamiento = [datos[i] for i in indices[:cantidad_entrenamiento]]
    datos_prueba = [datos[i] for i in indices[cantidad_entrenamiento:]]
    return datos_entrenamiento, datos_prueba

# Directorios donde se encuentran los datos
izquierda_dir = r"/content/DATA_JSON/IZQUIERDA_NORMALIZADO"
derecha_dir = r"/content/DATA_JSON/DERECHA_NORMALIZADO"

# Cargar los datos
datos_izquierda = cargar_datos(izquierda_dir)
datos_derecha = cargar_datos(derecha_dir)

# Dividir datos en conjuntos de entrenamiento y prueba
porcentaje_entrenamiento = 0.5
datos_entrenamiento_izquierda, datos_prueba_izquierda = dividir_datos(datos_izquierda, porcentaje_entrenamiento)
datos_entrenamiento_derecha, datos_prueba_derecha = dividir_datos(datos_derecha, porcentaje_entrenamiento)

# Convertir a arrays numpy
datos_entrenamiento_izquierda = np.array(datos_entrenamiento_izquierda)
datos_entrenamiento_derecha = np.array(datos_entrenamiento_derecha)
datos_prueba_izquierda = np.array(datos_prueba_izquierda)
datos_prueba_derecha = np.array(datos_prueba_derecha)

# Crear etiquetas para los datos de entrenamiento y prueba
etiquetas_entrenamiento = np.concatenate((np.ones(len(datos_entrenamiento_izquierda)), np.zeros(len(datos_entrenamiento_derecha))))
etiquetas_prueba = np.concatenate((np.ones(len(datos_prueba_izquierda)), np.zeros(len(datos_prueba_derecha))))

# Mezclar los datos de entrenamiento y prueba
datos_entrenamiento = np.concatenate((datos_entrenamiento_izquierda, datos_entrenamiento_derecha))
datos_prueba = np.concatenate((datos_prueba_izquierda, datos_prueba_derecha))

del datos_izquierda
del datos_derecha
del datos_entrenamiento_izquierda
del datos_entrenamiento_derecha
del datos_prueba_izquierda
del datos_prueba_derecha

In [ ]:
# Calcular la distribución de clases en los datos de entrenamiento y prueba
def calcular_distribucion_clases(etiquetas):
    clases, recuento = np.unique(etiquetas, return_counts=True)
    distribucion = dict(zip(clases, recuento))
    total_muestras = len(etiquetas)
    for clase, count in distribucion.items():
        porcentaje = (count / total_muestras) * 100
        print(f"Clase {clase}: {count} muestras ({porcentaje:.2f}%)")

# Calcular distribución de clases en los datos de entrenamiento
print("Distribución de clases en datos de entrenamiento:")
calcular_distribucion_clases(etiquetas_entrenamiento)

# Calcular distribución de clases en los datos de prueba
print("\nDistribución de clases en datos de prueba:")
calcular_distribucion_clases(etiquetas_prueba)

print(f"Número de muestras en datos de entrenamiento: {len(datos_entrenamiento)}")
print(f"Número de muestras en datos de prueba: {len(datos_prueba)}")

print(datos_entrenamiento[25])


## MODELO

MODELO ORIG

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import binary_crossentropy

# Definir el modelo
def define_model():
    model = Sequential()
    model.add(LSTM(64, return_sequences=True, input_shape=(100, 150)))
    model.add(Dropout(0.2))
    model.add(LSTM(64, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=Adam(), loss=binary_crossentropy, metrics=['accuracy'])
    return model

# Crear el modelo
model = define_model()




MODELO PRUEBA

### COMPILAR


In [ ]:
# Compilar el modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Resumen del modelo
model.summary()

## ENTRENAMIENTO

ORIG

In [ ]:
# Define your data generator function
def generador_datos(datos, etiquetas, batch_size):
    while True:
        indices = np.arange(len(datos))
        np.random.shuffle(indices)
        for i in range(0, len(datos), batch_size):
            batch_indices = indices[i:i + batch_size]
            batch_datos = [datos[j] for j in batch_indices]
            # Aplanar los datos de las poses y eliminar las dimensiones de tamaño 1
            batch_datos = np.array(batch_datos).reshape(-1, 100, 2*75)
            batch_etiquetas = np.array([etiquetas[j] for j in batch_indices])
            yield batch_datos, batch_etiquetas

# Define your parameters
batch_size = 5679  # Puedes ajustar esto según la memoria de tu máquina

# Crear generadores para los datos de entrenamiento y prueba
generador_entrenamiento = generador_datos(datos_entrenamiento, etiquetas_entrenamiento, batch_size)
generador_prueba = generador_datos(datos_prueba, etiquetas_prueba, batch_size)


In [ ]:
for x, y in generador_datos(datos_entrenamiento, etiquetas_entrenamiento, batch_size):
    print(x.shape)  # Debe imprimir algo como (batch_size, 100, 150)
    break

In [ ]:
steps_per_epoch=len(datos_entrenamiento) // batch_size
print(steps_per_epoch)

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
import datetime

# Directorio de registro para TensorBoard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Directorio para guardar el modelo
model_checkpoint_dir = "saved_models/"

# Definir callbacks para el entrenamiento, como detención temprana, TensorBoard y ModelCheckpoint
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5000, verbose=1, mode='min'),
    # Definir un ModelCheckpoint para guardar el modelo cada 100 épocas
    ModelCheckpoint(model_checkpoint_dir + "model_{epoch:02d}.h5",
                    save_best_only=False,  # No guardar solo el mejor modelo
                    save_freq=200,  # Guardar cada 100 épocas
                    verbose=1),
    tensorboard_callback
]

# Iniciar el entrenamiento con los callbacks definidos
history = model.fit(
    generador_entrenamiento,
    steps_per_epoch=len(datos_entrenamiento) // batch_size,
    epochs=100000,  # Número de épocas a entrenar
    validation_data=generador_prueba,
    validation_steps=len(datos_prueba) // batch_size,
    callbacks=callbacks  # Incluir todos los callbacks necesarios
)


In [ ]:
print(len(datos_entrenamiento) // batch_size)

3


### V2

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
import datetime
from keras.callbacks import Callback

class SaveModelEveryNthEpoch(Callback):
    def __init__(self, filepath, save_every_n_epochs):
        super(SaveModelEveryNthEpoch, self).__init__()
        self.filepath = filepath
        self.save_every_n_epochs = save_every_n_epochs

    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) % self.save_every_n_epochs == 0:
            filepath = self.filepath.format(epoch=epoch + 1)
            self.model.save(filepath)
            print(f"Model saved at epoch {epoch + 1}.")

# Directorio de registro para TensorBoard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)


# Directorio para guardar el modelo
model_checkpoint_dir = "saved_models2/"
model_checkpoint_path = model_checkpoint_dir + "model_{epoch:02d}.h5"

# Definir callbacks para el entrenamiento, incluyendo la nueva función de callback personalizada
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5000, verbose=1, mode='min'),
    SaveModelEveryNthEpoch(filepath=model_checkpoint_path, save_every_n_epochs=100),
    tensorboard_callback
]


# Iniciar el entrenamiento con los callbacks definidos
history = model.fit(
    generador_entrenamiento,
    steps_per_epoch=len(datos_entrenamiento) // batch_size,
    epochs=100000,  # Número de épocas a entrenar
    validation_data=generador_prueba,
    validation_steps=len(datos_prueba) // batch_size,
    callbacks=callbacks  # Incluir todos los callbacks necesarios
)




Epoch 1/100000
3/3 [==============================] - ETA: 0s - loss: 0.6932 - accuracy: 0.4994

ValueError: Unexpected value for `steps_per_epoch`. Received value is 0. Please check the docstring for `model.fit()` for supported values.

### Guardar modelo

In [ ]:
# Guardar el modelo
model.save("modelo_esgrima_v2.h5")

### Descargar

In [ ]:
from google.colab import files

zip -r modelo_esgrima_v2.zip modelo_esgrima_v2.h5
nombre_archivo = "modelo_esgrima_v2.zip"

files.download(nombre_archivo)

In [ ]:
from google.colab import files

zip -r modelo_esgrima_v2.zip modelo_esgrima_v2.h5
nombre_archivo = "modelo_esgrima_v2.zip"

files.download(nombre_archivo)

In [ ]:
import shutil
import os

# Carpeta que deseas comprimir
folder_to_zip = '/content/saved_models'

# Nombre del archivo zip que deseas crear
zip_filename = '/content/saved_models'

# Comprimir la carpeta en un archivo zip
shutil.make_archive(zip_filename, 'zip', folder_to_zip)

# Descargar el archivo zip
from google.colab import files
files.download(zip_filename)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## TENSORBOARD

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs/